In [2]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

# <span style="color:red;"><center> TD - Créer une page web dynamique</span>

Le but de ce TD est d'utiliser le module *Flask* de Python afin de **créer et de programmer un serveur web capable de générer des pages web dynamiques**.

## 1. Mise en place du serveur web

### 1.1 Générer une première page web

Dans votre dossier personnel, créer un répertoire *flask* et enregistrez-y le code Python ci-dessous dans un fichier *views.py*.
```python
    from flask import Flask, render_template, request
    import datetime


    app = Flask(__name__)

    @app.route('/')
    def index():
      return "<p>Tout fonctionne parfaitement</p>"

    app.run(debug=True)
```

Ouvrir le fichier *views.py* dans Pyzo et **l'exécuter comme un script (Ctrl+Shift+E)**. Dans le shell, vous devriez voir ceci :

     * Serving Flask app "views" (lazy loading)
     * Environment: production
       WARNING: This is a development server. Do not use it in a production deployment.
       Use a production WSGI server instead.
     * Debug mode: on
     * Restarting with stat
     * Debugger is active!
     * Debugger PIN: 196-352-360
     * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)

**Ouvrir un navigateur** et taper http://127.0.0.1:5000/ dans la **barre d'adresse**.

Une page web blanche avec écrit "Tout fonctionne parfaitement" devrait s'afficher.


### 1.2 Explications

La **première ligne** du code Python (en dehors des imports) **et la dernière**, permettent de **créer un serveur web local** (*localhost*), c'est-à-dire un serveur qui **tourne sur notre propre machine** au lieu d'être sur une autre machine accessible sur le web comme c'est le cas habituellement. Cela n'a d'intérêt que dans une **phase de test** ou dans un **but pédagogique** comme ici.

Son adresse IP est 127.0.0.1 et il fonctionne avec le protocole HTTP sur le port 5000.

La ligne `@app.rout('/')` est un décorateur (notion pas vue en NSI) indique que la fonction qui vient juste après doit être exécutée lorsqu'une requête demande à accéder à la racine "/" du serveur.

La fonction `index()` en question se contente de renvoyer le code HTML `<p>Tout fonctionne parfaitement</p>`.

Par conséquent, lorsque vous avez entré http://127.0.0.1:5000/ dans le navigateur, vous avez envoyé une requête HTTP sur le port 5000 au serveur web local pour accéder à sa racine et il vous avez renvoyé le code HTML convenu.

**Remarque :** En réalité, si vous regardez le code HTML correspondant à la page en tapant F12, vous verrez qu'il y a un peu plus que ça et notamment les balises *html*, *head* et *body*.

### 1.3 Générer une deuxième page web

Selon le même modèle, ajouter à *views.py* une fonction `about()` qui génerera une page web contenant le code HTML `p>Une autre page</p>` si on demande à accéder à l'URL http://127.0.0.1:5000/about.

Après avoir **redémarrer le serveur web**, en **exécutant à nouveau le programme Python comme un script** (normalement, sauvegarder le fichier *views.py* suffit à redémarrer le serveur), **vérifier que cela fonctionne** en tapant l'adresse ci-dessus dans la barre d'adresse.

### 1.4 Afficher une page web statique

Evidemment, il est aussi possible (et souvent préférable) de pouvoir directement **demander au serveur de renvoyer une page HTML existante** (web statique).

Pour cela, commencer par créer dans le répertoire *flask* un sous-répertoire *templates*. Nous allons voir que nous pouvons alors très facilement **demander au serveur de renvoyer les fichiers HTML placés dans** *templates* (et seulement ceux-là).  
Autre avantage, le fait de lancer le serveur en mode *debug* (`debug=True` à la dernière ligne de *views.py*) nous autorise à **prendre en compte les modifications du contenu de ces fichiers HTML sans redémarrer le serveur** (c'est-à-dire sans exécuter à nouveau le fichier Python).


Dans *templates*, **créer un fichier** *index.html* contenant le code HTML suivant :
```html
    <!doctype html>
    <html lang="fr">
        <head>
            <meta charset="utf-8">
            <title>Ma page</title>
        </head>
        <body>
          <h1>Mon super site</h1>
          <p>Tout fonctionne parfaitement</p>
        </body>
    </html>
```

Dans le fichier *views.py*, **modifier la fonction** `index()` pour qu'elle retourne ceci :

    return render_template("index.html")

**Relancer encore une fois le serveur** et allez à http://127.0.0.1:5000/ pour **vérifier** que la page *index.html* fonctionne bien.

**Modifier légèrement le fichier HTML** *index.html* (ajouter une mot ou faire une faute d'orthographe), **sauvegarder** et **rafraichir la page** dans le navigateur pour vérifier que la **modification est bien prise en compte sans relancer le serveur**.

## 2. Générer une page web dynamique

### 2.1 Ecrire du code HTML "dynamique"

**Remplacer** le code HTML de *index.html* par le code suivant :

**ATTENTION :** Les **accolades doivent être remplacées par des doubles accolades** mais Jupyter ne permet pas de les doubler sans que ça donne une erreur dans ce contexte.
```html
<!doctype html>
<html lang="fr">
	<head>
		<meta charset="utf-8">
		<title>Utilisation de Flask</title>
	</head>
	<body>
	  <h1>Mon super site</h1>
	  <p>Le serveur fonctionne parfaitement, il est {heure} h {minute} minutes et {seconde} secondes</p>
	</body>
</html>
```

Les mots "heure", "minute" et "seconde" entre **doubles accolades** signifient qu'il s'agit de variables qu'il faut remplacer par leur valeur.

### 2.2 Générer le code HTML dynamiquement

Il faut à présent **modifier** la fonction `index()` du fichier *views.py* de la façon suivante :

```python
def index():
  date = datetime.datetime.now()
  h = date.hour
  m = date.minute
  s = date.second
  return render_template("index.html", heure = h, minute = m, seconde = s)
```

Grâce au module `datetime`, les variables `h`, `m` et `s` contiennent les nombres correspondant aux heures, aux minutes et aux secondes. Ces valeurs sont ensuite données en argument à la fonction `render_template` afin de remplacer les variables `heure`, `minute` et `seconde` dans le fichier HTML.

**Rafraîchir la page HTML** (normalement, il n'est pas nécessaire de redémarrer le serveur) et celle-ci devrait à présent **afficher l'heure à la seconde près**.

**Rafraichissez la page après quelques secondes** et observez que l'heure a bien changé en conséquence.

### 2.3 Jinja (non, je n'ai pas dit ninja)

En fait, le fichier *index.html* ne contient **plus vraiment du code écrit en HTML mais dans le langage Jinja** qui sert à définir des *template HTML*.

En effet, bien que ce code ressemble beaucoup à du HTML, les **doubles accolades n'existent pas en HTML**. C'est la fonction `render_template` qui, à partir de ce code Jinja, va renvoyer un vrai code HTML qui sera ensuite inclus dans la réponse HTTP du serveur.

On donc bien du **web dynamique** puisque c'est seulement lorsque l'on envoie une **requête HTTP** (on rafraîchit la page) que la fonction `index()` est appelée et, à travers elle la fonction `render_template`, **créant ainsi le code HTML à partir du template** *index.html*.

Dans notre **exemple très simple**, le code HTML envoyé par le serveur sera presque identique au code Jinja du template. En effet, la seule différence est que les doubles accolades seront remplacées par des valeurs numériques. Mais évidemment, on **peut faire beaucoup plus selon le même principe**.

**Remarque :** La **notion de template** se rencontre assez souvent en informatique. Un template désigne en général un fichier qui est comme une sorte de coquille (à moitié) vide. En remplissant un même template, on peut alors **générer différents fichiers qui partagent la même structure générale mais avec un contenu différent**.

## 3. Formulaire d'une page web

Voyons à présent comme utiliser tout cela dans le cadre de **formulaires sur le web**.

### 3.1 Créer un formulaire HTML

**Remplacer** le code HTML de *index.html* par le code suivant :

```html
<!doctype html>
	<html lang="fr">
		<head>
			<meta charset="utf-8">
			<title>Le formulaire</title>
		</head>
		<body>
			<form action="http://localhost:5000/resultat" method="post">
					<label>Nom</label> : <input type="text" name="nom" />
					<label>Prénom</label> : <input type="text" name="prenom" />
					<input type="submit" value="Envoyer" />
			</form>
		</body>
	</html>
```

La balise `<form>` permet de créer un **élément de type formulaire** dans lequel nous avons placé **deux entrées** (`<input>`) de type *text* pour entrer un **nom** et un **prénom** et **une entrée** de type *submit* qui va créer un **bouton Envoyer**.

Les attributs `action ="http://localhost:5000/resultat"` et `method="post"` de la balise `<form>`indiquent **ce qu'il faut faire lorsque l'utilisateur clique** sur *Envoyer*. En l'occurence, il s'agira d'envoyer une requête HTTP avec la méthode *POST* vers l'URL "http://localhost:5000/resultat" (*localhost* est équivalent à l'adresse IP 127.0.0.1).

Pour que cela fonctionne cette **requête fournira dans son corps les chaines de caractères entrées** au préalables dans les champs *Nom* et *Prénom*.

En l'état, si vous **cliquez sur le bouton** *Envoyer*, vous devriez obtenir une jolie erreur *404 NOT FOUND* puisque l'URL pointe vers une ressource *resultat* qui **n'existe pas encore**.

### 3.2 Créer une page web utilisant les données du formulaire

Pour aller plus loin, nous devons créer un nouveau template qui permettra de générer une page web à partir des données entrées dans le formulaire.

Dans le dossier *templates*, créez un nouveau fichier *resultat.html* contenant le code Jinja suivant :
**ATENTION :** Il faut toujours remplacer les accolades par des doubles accolades.
```html
<!doctype html>
	<html lang="fr">
		<head>
			<meta charset="utf-8">
			<title>Résultat</title>
		</head>
		<body>
			<p>Bonjour {prenom} {nom}, j'espère que vous allez bien.</p>
		</body>
	</html>
```

Modifier le fichier *views.py* pour y **ajouter un nouveau décorateur et sa fonction associée** `resultat()` pour dire au serveur quoi faire que la ressource */resultat* est demandée :

```python
@app.route('/resultat',methods = ['POST'])
def resultat():
  result = request.form
  n = result['nom']
  p = result['prenom']
  return render_template("resultat.html", nom=n, prenom=p)
```

**Rafraichissez** la page http://127.0.0.1:5000/ pour afficher le formulaire, **remplissez-le** et **cliquez sur Envoyer**. Vous devriez être redirigé vers la page *resultat.html* qui a la politesse de vous **saluer par votre nom et votre prénom**.

### 3.3 Explications

Lors de la **première requête HTTP** vers la racine "/" du serveur (URL http://127.0.0.1:5000/), celui-ci génère (à partir du template *index.html*) la **page qui contient le formulaire** et renvoie le code HTML correspondant à votre navigateur (le client).

Ensuite, lorsque vous cliquez sur *Envoyer*, le navigateur envoie une **nouvelle requête** vers la ressource "/resultat" du serveur (URL http://localhost:5000/resultat)  avec la méthode POST et en fournissant le nom et le prénom entrés dans le formulaire. Cela génère, à partir du template *resultat.html* un **nouveau code HTML** qui est envoyé au navigateur et **finalement affiché**.

**Remarque :** Quand la méthode n'est pas précisée dans le décorateur `@app.route()` c'est GET qui est utilisée par défaut.

Pour **générer la page de résultat**, la fonction Python `resultat()` s'appuie sur `request.form` qui, à partir de la requête HTML envoyée avec la méthode *POST*, construit un **dictionnaire dont les clés sont les noms des champs du formulaire** (attributs `name` des balises `<input>`) et **les valeurs sont le contenu de ces champs** (ce qui est entré par  l'utilisateur).

Il ne reste ensuite plus qu'à **appeler la fonction** `render_template()` en lui **fournissant en argument le template** *resultat.html* et **les valeurs des clés** *nom* et *prenom* du dictionnaire.

### 3.4 Utiliser *GET* à la place de *POST*

Comme on l'a dit dans la partie cours, il est en général **possible d'utiliser la méthode** *GET* **à la place de** *POST* pour envoyer de l'information au serveur.

Pour cela, il suffit :
- de remplacer l'attribut `method='post'` par `method='get'` pour la balise `<form>` du fichier *index.html*,
- de remplacer `methods=['POST']`par `methods=['GET']` et `result=request.form` par `result=request.args` dans le fichier *views.py*.

**Tester** pour vérifier que cela fonctionne toujours.

Le **résultat est le même mais pas l'URL** qui s'affiche dans la barre d'adresse. Si vous portez une salopette bleue, une casquette rouge et que vous arborez une moustache bien fournie, vous pourriez y voir quelque chose comme :
http://localhost:5000/resultat?nom=Bros&prenom=Mario

Ainsi, vous voyez qu'ici les **données entrées dans le formulaire ont été placées directement dans l'URL** utilisée par la méthode *GET*. Ce n'était pas le cas avec la méthode *POST*.

Ce n'est évidemment **pas une bonne pratique lorsque les données sont sensibles** car elles sont alors accessibles par n'importe qui, même avec le protocole HTTPS (je crois).

D'ailleurs, **remplacez** *Mario* par *Luigi* **dans la barre d'adresse** puis tapez sur **Entrée**. Voilà que vous venez d'usurper l'identité de votre frère jumeau !

### 3.5 Observer les requêtes HTTP

Pour terminer, sachez que l'on peut **très facilement visualiser les requêtes HTTP échangées entre le navigateur et le serveur**.

Sur la **plupart des navigateurs**, il suffit pour cela d'appuyer sur la **touche** *F12* (ou *clic droit* puis *Inspecter*) puis de choisir l'**onglet** *Network* (ou *Réseau* en Français).

**Faites-le** dans votre navigateur lorsque vous êtes sur la **page de résultat**, puis **rafraichissez la page**. Vous verrez alors s'afficher la **requête avec la méhode** *GET* et le statut *200*.

Si maintenant vous **revenez à la version précédente** de *views.py* et *index.html*, lorque la méthode *POST* était utilisée. Puis que vous **revenez au formulaire**, que vous le **rafraichissez** et que vous le **remplissez à nouveau**, vous devriez constater que c'est bien une **requête avec la méthode** *POST* qui apparaît cette fois.